# Skinning

In [2]:
import sys
import cv2
import matplotlib.pyplot as plt
from tf_pose.common import read_imgfile
from lib.common import draw_circle
from lib.contour import find_human_contour
from lib.skeleton import SkeletonImplement, SkeletonTest
from lib.skinning import Skinning

In [3]:
src = read_imgfile("./images/shadow.jpg", None, None)
dst = src.copy()
human_contour = find_human_contour(src)

skeletonImplement = SkeletonImplement()
human = skeletonImplement.infer_skeleton(src)
human = skeletonImplement.remove_unused_joints(human)

skeletonTest = SkeletonTest(human, human_contour, src.shape)
# skeletonTest.report()
if not skeletonTest.is_reliable():
    print("This skeleton model is not reliable.")
    sys.exit(0)

skinning = Skinning(src, human, human_contour, algorithm="nearest_neighbour_within_contour")

[2018-10-26 22:37:00,512] [TfPoseEstimator] [INFO] loading graph from C:\Users\kukikeita\Desktop\Python\ShadowSkinning\tf_pose_data\graph/mobilenet_thin/graph_opt.pb(default size=368x368)


In [4]:
vertices = skinning.contour_vertex_positions
vertices = [[int(x[0]), int(x[1])] for x in vertices]
triangles = skinning.triangle_vertex_indices
nearest_joints = skinning.nearest_body_part_indices
influence = skinning.influence
joints = skinning.body_part_positions
joints = [[x[0], x[1]] for x in joints]

display(len(vertices))
display(len(triangles)*3)
display(len(nearest_joints))
display(len(influence))
display(len(joints))

611

1395

611

611

18

# OSC

In [5]:
from pythonosc import udp_client
from time import sleep

In [6]:
ip = "127.0.0.1"
port = 5005

In [7]:
client = udp_client.SimpleUDPClient(ip, port)

In [8]:
# address
vertices_address = "/vertices"
triangles_address = "/triangles"
nearest_joints_address = "/nearest_joints"
influence_address = "/influence"
joints_address = "/joints"
end_address = "/end"

In [9]:
# 軽い順で送らないと後半が欠落する可能性あり
# 安全のためsleepいれとく（要修正）
client.send_message(joints_address, joints)
sleep(0.1)
client.send_message(influence_address, influence)
sleep(0.1)
client.send_message(nearest_joints_address, nearest_joints)
sleep(0.1)
client.send_message(triangles_address, triangles)
sleep(0.1)
client.send_message(vertices_address, vertices)
sleep(0.1)
client.send_message(end_address, "end")